# テンソル多体分解の実装


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools

import numpy as np

from LegendreDecomposition import LD as LD_cupy
from LegendreDecomposition.old import LD

# 実験


## Random テンソルによる実験


### ２次近似


In [3]:
N = 10
X = np.random.uniform(0, 0.1, size=(N, N, N, N))
X

array([[[[0.02456185, 0.07594729, 0.04632217, ..., 0.07911832,
          0.04749621, 0.00108574],
         [0.01785412, 0.0186851 , 0.02473015, ..., 0.06143376,
          0.06415897, 0.01579581],
         [0.0193518 , 0.06048574, 0.09015333, ..., 0.01439587,
          0.07400735, 0.07780935],
         ...,
         [0.05881657, 0.06845316, 0.05433782, ..., 0.03826952,
          0.09360572, 0.05893505],
         [0.0669766 , 0.0986031 , 0.0405482 , ..., 0.04040596,
          0.02977316, 0.06650399],
         [0.04605704, 0.08201341, 0.04559612, ..., 0.00542682,
          0.06055674, 0.01441761]],

        [[0.02331532, 0.03095989, 0.09264013, ..., 0.038823  ,
          0.08483852, 0.09208708],
         [0.03489452, 0.04178505, 0.05531045, ..., 0.06053565,
          0.0524742 , 0.05791717],
         [0.09309412, 0.03150101, 0.02757887, ..., 0.08750837,
          0.02062332, 0.038026  ],
         ...,
         [0.03883839, 0.09601238, 0.05670824, ..., 0.03814517,
          0.0906253 , 0.0

In [91]:
%timeit all_history_kl, scaleX, Q, Hq = LD(X, order=2, verbose=False)

502 ms ± 4.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit LD_cupy(X, order=2, gpu=False, verbose=False)

32 ms ± 4.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%timeit LD_cupy(X, order=2, verbose=False)

24.3 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit LD_cupy(X, order=2, verbose=False, dtype=np.float32)

13.7 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 画像テンソルによる実験


### 2 次近似


In [8]:
from pathlib import Path

import matplotlib.pyplot as plt
from PIL import Image

X = []
for k in range(1, 5):
    # ファイル名、パス
    file_name = Path("..") / "tests" / "resources" / "coil-100/obj{}__0.png".format(k)
    img = Image.open(file_name)
    img = img.resize((32, 32))
    array_obj = np.asarray(img)
    # print(array_obj.shape)
    # 画像の表示
    # plt.imshow(array_obj)
    # plt.show()
    X.append(array_obj)
X = np.array(X)
X = (X + 1) / 256.0
X.shape

FileNotFoundError: [Errno 2] No such file or directory: '/home/giprayogo/work/manybodytensor/tests/resources/coil-100/obj1__0.png'

In [105]:
%timeit all_history_kl, scaleX, Q, Hq = LD(X, order=2, verbose=False)

11.8 s ± 21.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [106]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(X, order=2, gpu=False, verbose=False)

368 ms ± 12.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [107]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(X, order=2, verbose=False)

110 ms ± 968 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [108]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(X, order=2, dtype=np.float32, verbose=False)

74.3 ms ± 271 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### リング分解による実装


In [109]:
# ring decomposition
def constraint(I):
    I = np.array(I)
    idx = np.where(I)[0]
    if len(idx) == 2:
        if idx[1] - idx[0] == 1 or (idx[1] == 3 and idx[0] == 0):
            return True
        else:
            return False
    else:
        return False


B = [
    I
    for I in itertools.product(*[list(range(X.shape[d])) for d in range(len(X.shape))])
    if constraint(I)
]

In [110]:
%timeit all_history_kl, scaleX, Q, Hq = LD(X, B, verbose=False)

3.4 s ± 5.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [111]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(X, B, gpu=False, verbose=False)

107 ms ± 5.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [112]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(X, B, verbose=False)

21.1 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [113]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(X, B, verbose=False, dtype=np.float32)

8.41 ms ± 8.14 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 依存関係 B を設計


In [116]:
# ２次の近似から、index - channel 結合を切断する
def constraint(I):
    I = np.array(I)
    if np.sum(I != 0) <= 2:
        if I[0] != 0 and I[3] != 0:
            return False
        else:
            return True
    else:
        return False


B = [
    I
    for I in itertools.product(*[list(range(X.shape[d])) for d in range(len(X.shape))])
    if constraint(I)
]

In [117]:
%timeit all_history_kl, scaleX, Q, Hq = LD(X, B, verbose=False)

9.56 s ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [118]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(X, B, gpu=False, verbose=False)

282 ms ± 60.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [119]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(X, B, verbose=False)

49.9 ms ± 37.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [120]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(X, B, verbose=False, dtype=np.float32)

18.2 ms ± 6.15 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## MovieLens データセットによる実験


In [121]:
import pandas as pd

df = pd.read_csv("ml-latest-small/movies.csv")

genres_set = set()
for i, r in df.iterrows():
    for el in r["genres"].split("|"):
        genres_set.add(el)
genres_list = list(genres_set)

In [122]:
edges_mg_dict = {}
mid_dict = {}
for i, r in df.iterrows():
    for el in r["genres"].split("|"):
        genres_set.add(el)
        j = genres_list.index(el)
        m = int(r["movieId"])
        if m not in mid_dict:
            mid_dict[m] = len(mid_dict)
        if mid_dict[m] not in edges_mg_dict:
            edges_mg_dict[mid_dict[m]] = []
        edges_mg_dict[mid_dict[m]].append(j)

In [123]:
df = pd.read_csv("ml-latest-small/ratings.csv")
edges = []

for i, r in df.iterrows():
    m = int(r["movieId"])
    if m in mid_dict:
        for g in edges_mg_dict[mid_dict[m]]:
            e = (int(r["userId"]) - 1, mid_dict[m], g, r["rating"])
            edges.append(e)

In [124]:
u_max = max([e[0] for e in edges])
m_max = max([e[1] for e in edges])
g_max = max([e[2] for e in edges])

print(u_max, m_max, g_max)

609 9741 19


In [125]:
import numpy as np

M = np.zeros((u_max + 1, m_max + 1, g_max + 1))
for u, m, g, r in edges:
    M[u, m, g] = r

In [126]:
M_ = M[:10, :10, :]

In [127]:
M_.shape

(10, 10, 20)

In [128]:
%timeit all_history_kl, scaleX, Q, Hq = LD(M_, order=2, eps=1.0e-10, verbose=False)

416 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [129]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(M_, order=2, eps=1.0e-10, gpu=False, verbose=False)

32.2 ms ± 2.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [130]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(M_, order=2, eps=1.0e-10, verbose=False)

13.8 ms ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [131]:
%timeit all_history_kl, scaleX, Q, Hq = LD_cupy(M_, order=2, eps=1.0e-10, verbose=False, dtype=np.float32)

11.8 ms ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## larger and larger random tensor


In [12]:
N = 10
X = np.random.uniform(0, 0.1, size=(N, N, N, N))

In [13]:
%timeit LD(X, order=2, verbose=False)
%timeit LD_cupy(X, order=2, gpu=False, verbose=False)
%timeit LD_cupy(X, order=2, verbose=False)

504 ms ± 7.38 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
22.5 ms ± 304 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
7.04 ms ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
N = 12
X = np.random.uniform(0, 0.1, size=(N, N, N, N))

In [15]:
%timeit LD(X, order=2, verbose=False)
%timeit LD_cupy(X, order=2, gpu=False, verbose=False)
%timeit LD_cupy(X, order=2, verbose=False)

1.02 s ± 747 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
52.2 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
9.14 ms ± 22.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
N = 14
X = np.random.uniform(0, 0.1, size=(N, N, N, N))

In [17]:
%timeit all_history_kl, scaleX, Q, Hq = LD(X, order=2, verbose=False)
%timeit LD_cupy(X, order=2, gpu=False, verbose=False)
%timeit LD_cupy(X, order=2, verbose=False)

2.74 s ± 4.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
134 ms ± 39.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
19.2 ms ± 51.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
N = 17
X = np.random.uniform(0, 0.1, size=(N, N, N, N))

In [19]:
%timeit all_history_kl, scaleX, Q, Hq = LD(X, order=2, verbose=False)
%timeit LD_cupy(X, order=2, gpu=False, verbose=False)
%timeit LD_cupy(X, order=2, verbose=False)

8.08 s ± 31.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
305 ms ± 7.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
40.8 ms ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
N = 20
X = np.random.uniform(0, 0.1, size=(N, N, N, N))

In [22]:
%timeit all_history_kl, scaleX, Q, Hq = LD(X, order=2, verbose=False)
%timeit LD_cupy(X, order=2, gpu=False, verbose=False)
%timeit LD_cupy(X, order=2, verbose=False)

31.1 s ± 107 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.18 s ± 66.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
140 ms ± 21.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
X = np.random.uniform(0, 0.1, size=(30, 30, 30, 30, 30))
X.shape

(30, 30, 30, 30, 30)

In [43]:
all_history_kl, scaleX, Q, Hq = LD_cupy(X, order=2, verbose=True)

iter= 0 kl= 0.19309474471629862 mse= 5.643692895071144e-16
iter= 1 kl= 237.6959260464662 mse= 1.000005641734787e-10
iter= 2 kl= 237.69592604453695 mse= 1.0000056417348576e-10


In [44]:
X = np.random.uniform(0, 0.1, size=(20, 20, 20, 20, 20, 20))
X.shape

(20, 20, 20, 20, 20, 20)

In [45]:
all_history_kl, scaleX, Q, Hq = LD_cupy(X, order=2, verbose=True)

iter= 0 kl= 0.19300479988028052 mse= 8.132596317528895e-17
iter= 1 kl= 633.7299752973898 mse= 1.0000008131882626e-10
iter= 2 kl= 633.729975297318 mse= 1.0000008131882638e-10
